In [1]:
import ee
import os
import matplotlib.pyplot as plt
import pickle

ee.Authenticate()
os.environ['HTTP_PROXY']='http://127.0.0.1:7890'
os.environ['HTTPS_PROXY']='http://127.0.0.1:7890'
ee.Initialize()


Successfully saved authorization token.


In [2]:
#边界
guangxi_cities = ee.FeatureCollection("projects/chancy479/assets/city2")

cities = [
    "南宁市",
    "柳州市",
    "桂林市",
    "梧州市",
    "北海市",
    "防城港市",
    "钦州市",
    "贵港市",
    "玉林市",
    "百色市",
    "贺州市",
    "河池市",
    "来宾市",
    "崇左市"
]

Roi = []
for i in range(len(cities)):
    Reigon_city = guangxi_cities.filterMetadata('市','equals',cities[i])
    Roi.append(Reigon_city)


In [37]:
def lst_mean(image,Roi):
        # image=image.select('LST_Day_1km_mean')
        mean = image.reduceRegion(reducer=ee.Reducer.mean(), scale=30, geometry=Roi,bestEffort=True)#计算所有波段的平均值
        print(mean.getInfo())
        value = mean.getInfo().get('LST_Day_1km_mean')
        print(value)
        return value

In [40]:
def get_values_monthly(Roi):

    # 1.循环获取数据集
    years=[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    months=['01','02','03','04','05','06','07','08','09','10','11','12']
    # 初始化一个空列表，用于存储按月划分的数据集a
    monthly_datasets = []
    for i in range(len(years)-1):
            for j in range(1,13):
                if(j==12):
                    start_date=str(years[i]) + '-' + '12-01'
                    end_date=str(years[i+1]) + '-' + '01-01'
                    filtered=ee.ImageCollection('MODIS/061/MOD11A1')\
                        .filterBounds(Roi)\
                        .filterDate(start_date, end_date)\
                        .select('LST_Day_1km')
                else:
                    start_date=str(years[i]) + '-' + months[j-1] + '-01'
                    end_date=str(years[i]) + '-' + months[j]+ '-01'
                    filtered=ee.ImageCollection('MODIS/061/MOD11A1')\
                        .filterBounds(Roi)\
                        .filterDate(start_date, end_date)\
                        .select('LST_Day_1km')
                if(start_date != '2000-01-01'):
                  monthly_datasets.append(filtered)
                # else:
                #   print(start_date,end_date)
    print(len(monthly_datasets))
    # for i in range(len(monthly_datasets)):
    #   print(monthly_datasets[i].size().getInfo())

    #2.将每年的imagecollection reduce 成一张平均值image
    mean_values = []
    for i in range(len(monthly_datasets)):

        meanImage1 = monthly_datasets[i].reduce(ee.Reducer.mean())
        print(meanImage1.getInfo())
        print(str(i))
        value = lst_mean(meanImage1,Roi) 
        mean_values.append(value)
        print('ok!')

    return mean_values




In [41]:

lst_values_cities =[]
for i in range(9,len(Roi)):
    print(cities[i])
    lst=get_values_monthly(Roi[i])
    print('finish'+cities[i])
    print(lst)
    with open('LST_values'+str(i)+'.pkl', 'wb') as f:
        pickle.dump(lst, f)
    lst_values_cities.append(lst)

with open('LST_values_all_cities.pkl', 'wb') as f:
      pickle.dump(lst_values_cities, f)

百色市
251
{'type': 'Image', 'bands': [{'id': 'LST_Day_1km_mean', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
0
{'LST_Day_1km_mean': None}
None
ok!
{'type': 'Image', 'bands': [{'id': 'LST_Day_1km_mean', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
1
{'LST_Day_1km_mean': 14968.057984649}
14968.057984649
ok!
{'type': 'Image', 'bands': [{'id': 'LST_Day_1km_mean', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
2
{'LST_Day_1km_mean': 15104.474274540216}


KeyboardInterrupt: 